# Single Human Target Locking

In [1]:
import cv2

TrDict = {'csrt': cv2.TrackerCSRT_create,
         'kcf' : cv2.TrackerKCF_create,
         'boosting' : cv2.legacy.TrackerBoosting_create,
         'mil': cv2.TrackerMIL_create,
         'tld': cv2.legacy.TrackerTLD_create,
         'medianflow': cv2.legacy.TrackerMedianFlow_create,
         'mosse':cv2.legacy.TrackerMOSSE_create}

tracker = TrDict['csrt']()

v = cv2.VideoCapture('/Users/xy0ke/Downloads/multiple-object-tracking-in-videos-using-opencv-in-python-master/Youtube/Video 01/data/mot.mp4') # video
# v = cv2.VideoCapture(0)
v.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
v.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

ret, frame = v.read()
cv2.imshow('Frame', frame)

bb = cv2.selectROI('Frame', frame)
tracker.init(frame, bb)

while True:
    ret, frame = v.read()
    timer = cv2.getTickCount()
    if not ret:
        break

    (success,box) = tracker.update(frame)

    if success:
        (x,y,w,h) = [int(a) for a in box]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(100,255,0),2)

    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)
    cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2)

    cv2.imshow('Frame',frame)

    key = cv2.waitKey(5) & 0xFF
    if key == ord('q'):
        break

v.release()
cv2.destroyAllWindows()

2023-09-24 22:45:14.041 Python[7970:5117197] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/core/src/dxt.cpp:3508: error: (-215:Assertion failed) type == CV_32FC1 || type == CV_32FC2 || type == CV_64FC1 || type == CV_64FC2 in function 'dft'


# Multiple Human Target Locking

In [ ]:
# from __future__ import print_function
import sys
import cv2
from random import randint


trackerTypes = ['BOOSTING', 'MIL', 'KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']


def createTrackerByName(trackerType):
    # Create a tracker based on tracker name
    if trackerType == trackerTypes[0]:
        tracker = cv2.legacy.TrackerBoosting_create()
    elif trackerType == trackerTypes[1]:
        tracker = cv2.legacy.TrackerMIL_create()
    elif trackerType == trackerTypes[2]:
        tracker = cv2.legacy.TrackerKCF_create()
    elif trackerType == trackerTypes[3]:
        tracker = cv2.legacy.TrackerTLD_create()
    elif trackerType == trackerTypes[4]:
        tracker = cv2.legacy.TrackerMedianFlow_create()
    elif trackerType == trackerTypes[5]:
        tracker = cv2.legacy.TrackerGOTURN_create()
    elif trackerType == trackerTypes[6]:
        tracker = cv2.TrackerMOSSE_create()
    elif trackerType == trackerTypes[7]:
        tracker = cv2.legacy.TrackerCSRT_create()
    else:
        tracker = None
        print("Incorrect tracker name")
        print("Available trackers are:")
        for t in trackerTypes:
            print(t)

    return tracker


# Set video to load
videoPath = "/Users/xy0ke/Downloads/multiple-object-tracking-in-videos-using-opencv-in-python-master/Youtube/Video 01/data/mot.mp4"

# Create a video capture object to read videos
cap = cv2.VideoCapture(videoPath)

# Read first frame
success, frame = cap.read()
# quit if unable to read the video file
if not success:
    print("Failed to read video")
    sys.exit(1)

    ## Select boxes
bboxes = []
colors = []

# OpenCV's selectROI function doesn't work for selecting multiple objects in Python
# So we will call this function in a loop till we are done selecting all objects
while True:
    # draw bounding boxes over objects
    # selectROI's default behaviour is to draw box starting from the center
    # when fromCenter is set to false, you can draw box starting from top left corner
    bbox = cv2.selectROI("MultiTracker", frame)
    bboxes.append(bbox)
    colors.append((randint(0, 255), randint(0, 255), randint(0, 255)))
    print("Press q to quit selecting boxes and start tracking")
    print("Press any other key to select next object")
    k = cv2.waitKey(0) & 0xFF
    print(k)
    if k == 113:  # q is pressed
        break

print("Selected bounding boxes {}".format(bboxes))

# Specify the tracker type
trackerType = "CSRT"
createTrackerByName(trackerType)

# Create MultiTracker object
multiTracker = cv2.legacy.MultiTracker_create()


# Initialize MultiTracker
for bbox in bboxes:
    multiTracker.add(createTrackerByName(trackerType), frame, bbox)

    # Process video and track objects
while cap.isOpened():
    success, frame = cap.read()
    timer = cv2.getTickCount()
    if not success:
        break

    # get updated location of objects in subsequent frames
    success, boxes = multiTracker.update(frame)

    # draw tracked objects
    for i, newbox in enumerate(boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        cv2.rectangle(frame, p1, p2, colors[i], 2, 1)

    # show frame
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)
    cv2.putText(
        frame,
        "FPS : " + str(int(fps)),
        (100, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.75,
        (50, 170, 50),
        2,
    )
    cv2.imshow("MultiTracker", frame)

    # quit on ESC button
    if cv2.waitKey(1) & 0xFF == 27:  # Esc pressed
        break


# Multiple Human Target No Locking (YOLO v8)

In [ ]:
import cv2

from ultralytics import YOLO
import supervision as sv

frame_width, frame_height = [1280, 720]

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

model = YOLO("/Users/xy0ke/Desktop/YOLO v8 on mac m1/yolov8n.pt")

box_annotator = sv.BoxAnnotator(
    thickness=2,
    text_thickness=2,
    text_scale=1
)


while True:
    ret, frame = cap.read()

    if not ret:
        break

    result = model(frame, agnostic_nms=True, classes=[0])[0]
    detections = sv.Detections.from_yolov8(result)
    
    labels = [
        f"{model.model.names[class_id]} {round(confidence*100, 2)}%"
        for _, _, confidence, class_id, _ in detections
        
    ]
    frame = box_annotator.annotate(
        scene=frame, 
        detections=detections,
        labels=labels
    )     
    
    cv2.imshow("yolov8", frame)

    if (cv2.waitKey(30) == 27):
        break

cap.release()
cv2.destroyAllWindows()

# Human Targets with Trailing System No Locking (YOLO v8)

In [1]:
from collections import defaultdict

import cv2
import numpy as np

from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Open the video file
# video_path = "/Users/xy0ke/Desktop/YOLO v8 on mac m1/car.mp4"
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('/Users/xy0ke/Downloads/multiple-object-tracking-in-videos-using-opencv-in-python-master/Youtube/Video 01/data/mot.mp4') # video

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(source=frame, persist=True, classes=[0])

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Display the annotated frame
        cv2.imshow("Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if (cv2.waitKey(30) == 27):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 1 person, 124.9ms
Speed: 5.4ms preprocess, 124.9ms inference, 14.7ms postprocess per image at shape (1, 3, 384, 640)
2023-09-24 15:49:41.020 Python[4623:5014284] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.

0: 384x640 1 person, 188.9ms
Speed: 4.0ms preprocess, 188.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)


AttributeError: 'NoneType' object has no attribute 'int'